#### How to use this notebook?

```
conda activate deepFCD
pip install jupyterlab
git clone https://github.com/NOEL-MNI/deepFCD
cd deepFCD/app
```

- Using the GPU: cuda0
    ```
    THEANO_FLAGS=mode=FAST_RUN,device=cuda0,floatX=float32,dnn.enabled=False jupyter lab --no-browser .
    ```

- Using the GPU: cudaX [replace X with the desired GPU-ID]
    ```
    THEANO_FLAGS=mode=FAST_RUN,device=cudaX,floatX=float32,dnn.enabled=False jupyter lab --no-browser .
    ```

- Using the CPU [not recommended, runtimes can exceed several hours/days depending on the hardware]
    ```
    THEANO_FLAGS=mode=FAST_RUN,device=cpu,floatX=float32,dnn.enabled=False jupyter lab --no-browser .
    ```

In [1]:
import os
import sys
import multiprocessing
from config.experiment import options
import warnings
warnings.filterwarnings('ignore')
import time
import numpy as np
import setproctitle as spt
from tqdm import tqdm

os.environ["KERAS_BACKEND"] = "theano"

# deepFCD imports
from models.noel_models_keras import *
from keras.models import load_model
from keras import backend as K
from utils.metrics import *
from utils.base import *

# deepMask imports
import torch
from mo_dots import Data
from deepMask.app.utils.data import *
from deepMask.app.utils.deepmask import *
from deepMask.app.utils.image_processing import noelImageProcessor
import deepMask.app.vnet as vnet

Using Theano backend.
Can not use cuDNN on context None: Disabled by dnn.enabled flag
Mapped name None to device cuda0: NVIDIA TITAN RTX (0000:81:00.0)


Error: /tmp/qab_5801.log - No such file or directory.


### Directory Organization
```bash
/data
└── FCD_001/           # [this patient-specific directory is contained within ${IO_DIRECTORY}]
    ├── noel_deepFCD_dropoutMC  # [deepFCD output images]
    │   ├── FCD_001_noel_deepFCD_dropoutMC_prob_mean_0.nii.gz # [mean PROBABILITY image from CNN-1]
    │   ├── FCD_001_noel_deepFCD_dropoutMC_prob_mean_1.nii.gz # [mean PROBABILITY image from CNN-2]
    │   ├── FCD_001_noel_deepFCD_dropoutMC_prob_var_0.nii.gz  # [mean UNCERTAINTY image from CNN-1]
    │   └── FCD_001_noel_deepFCD_dropoutMC_prob_var_1.nii.gz  # [mean UNCERTAINTY image from CNN-2]
    ├── t1.nii.gz # [input #1: T1-weighted image]
    └── t2.nii.gz # [input #2: T2-weighted FLAIR image]
```

#### Note: The below configuration assumes T1 and T2 contrasts are in their native sterotaxic space, and therefore, will be preprocessed before brain extraction and FCD detection.
#### Preprocessing entails: 1) T1 and T1 co-registration to MNI template space, and 2) Non-homogeneity correction

### Configuration

In [2]:
# global configuration
args = Data()
args.dir = '/data'
args.id = 'FCD_001'
args.brain_masking = True # set to True or any non-zero value for brain extraction or skull-removal, False otherwise
args.preprocess = True # co-register T1 and T2 contrasts before brain extraction
args.outdir =  os.path.join(args.dir, args.id)
args.seed = 666

args.t1_fname = 't1.nii.gz'
args.t2_fname = 't2.nii.gz'
args.t1 = os.path.join(args.outdir, args.t1_fname)
args.t2 = os.path.join(args.outdir, args.t2_fname)
cwd = os.getcwd()

# deepFCD configuration
K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')  # TH dimension ordering in this code

options['parallel_gpu'] = False
modalities = ['T1', 'FLAIR']
x_names = options['x_names']
options['dropout_mc'] = True
options['batch_size'] = 350000
options['mini_batch_size'] = 2048
options['load_checkpoint_1'] = True
options['load_checkpoint_2'] = True

# trained model weights based on 148 histologically-verified FCD subjects
options['test_folder'] = args.dir
options['weight_paths'] = os.path.join(cwd, 'weights')
options['experiment'] = 'noel_deepFCD_dropoutMC'
spt.setproctitle(options['experiment'])
print("experiment: {}".format(options['experiment']))

experiment: noel_deepFCD_dropoutMC


In [3]:
if args.brain_masking:
    # trained weights based on manually corrected masks from
    # 153 patients with cortical malformations
    args.inference = os.path.join(cwd, 'deepMask/app/weights', 'vnet_masker_model_best.pth.tar')
    # resize all input images to this resolution matching training data
    args.resize = (160,160,160)
    args.use_gpu = False
    args.cuda = torch.cuda.is_available() and args.use_gpu
    torch.manual_seed(args.seed)
    args.device_ids = list(range(torch.cuda.device_count()))
    if args.cuda:
        torch.cuda.manual_seed(args.seed)
        print("build vnet, using GPU")
    else:
        print("build vnet, using CPU")
    model = vnet.build_model(args)
    template = os.path.join(cwd, 'deepMask/app/template', 'mni_icbm152_t1_tal_nlin_sym_09a.nii.gz')

    # MRI pre-processing configuration
    args.output_suffix = '_brain_final.nii.gz'
    
    noelImageProcessor(id=args.id, t1=args.t1, t2=args.t2, output_suffix=args.output_suffix, output_dir=args.outdir, template=template, usen3=True, args=args, model=model, preprocess=args.preprocess).pipeline()
else:
    print("Skipping image preprocessing and brain masking, presumably images are co-registered, bias-corrected, and skull-stripped")

Skipping image preprocessing and brain masking, presumably images are co-registered, bias-corrected, and skull-stripped


### Initialize the CNN, and load the model weights from disk

In [4]:
# initialize empty model
model = None
# initialize the CNN architecture
model = off_the_shelf_model(options)

load_weights = os.path.join(options['weight_paths'], 'noel_deepFCD_dropoutMC_model_1.h5')
print("loading DNN1, model[0]: {} exists".format(load_weights)) if os.path.isfile(load_weights) else sys.exit("model[0]: {} doesn't exist".format(load_weights))
model[0] = load_model(load_weights)

load_weights = os.path.join(options['weight_paths'], 'noel_deepFCD_dropoutMC_model_2.h5')
print("loading DNN2, model[1]: {} exists".format(load_weights)) if os.path.isfile(load_weights) else sys.exit("model[1]: {} doesn't exist".format(load_weights))
model[1] = load_model(load_weights)
print(model[1].summary())

loading DNN1, model[0]: /local_raid/data/ravnoor/02_docker/deepFCD/app/weights/noel_deepFCD_dropoutMC_model_1.h5 exists
loading DNN2, model[1]: /local_raid/data/ravnoor/02_docker/deepFCD/app/weights/noel_deepFCD_dropoutMC_model_2.h5 exists
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2, 16, 16, 16)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 48, 16, 16, 16)    2640      
_________________________________________________________________
batch_normalization_3 (Batch (None, 48, 16, 16, 16)    192       
_________________________________________________________________
dropout_4 (Dropout)          (None, 48, 16, 16, 16)    0         
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 48, 8, 8, 8)       0         
__________________________________

### Test the trained CNN model to get predictions

In [5]:
test_list = [args.id]

files = [args.t1, args.t2]

test_data = {}
test_data = {f: {m: os.path.join(options['test_folder'], f, n) for m, n in zip(modalities, files)} for f in test_list}

for _, scan in enumerate(tqdm(test_list, desc='serving predictions using the trained model', colour='blue')):
    t_data = {}
    t_data[scan] = test_data[scan]

    options['pred_folder'] = os.path.join(options['test_folder'], scan, options['experiment'])
    if not os.path.exists(options['pred_folder']):
        os.mkdir(options['pred_folder'])

    pred_mean_fname = os.path.join(options['pred_folder'], scan + '_prob_mean_1.nii.gz')
    pred_var_fname = os.path.join(options['pred_folder'], scan + '_prob_var_1.nii.gz')

    if np.logical_and(os.path.isfile(pred_mean_fname), os.path.isfile(pred_var_fname)):
        print("prediction for {} already exists".format(scan))
        continue

    options['test_scan'] = scan

    start = time.time()
    print('\n')
    print('-'*70)
    print("testing the model for scan: {}".format(scan))
    print('-'*70)

    test_model(model, t_data, options)

    end = time.time()
    diff = (end - start) // 60
    print("-"*70)
    print("time elapsed: ~ {} minutes".format(diff))
    print("-"*70)

serving predictions using the trained model:   0%|                                                                                     | 0/1 [00:00<?, ?it/s]



----------------------------------------------------------------------
testing the model for scan: PX_012_2
----------------------------------------------------------------------



predict_stochastic: 100%|####################################################################################################| 20/20 [12:30<00:00, 37.53s/it]

predict_stochastic: 100%|####################################################################################################| 20/20 [12:30<00:00, 37.53s/it]

predict_stochastic: 100%|#################################################################################################################################################################################################################################| 20/20 [03:51<00:00, 11.60s/it]

serving predictions using the trained model: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [50:22<00:00, 3022.59s/it]

----------------------------------------------------------------------
time elapsed: ~ 50.0 minutes
----------------------------------------------------------------------
